#  Cascade Refrigeration Cycle 

## 1 Cascade Refrigeration Cycle

>>Majid Bahrami. ENSC461(S11)：Energy Conversion,Simon Fraser University,Canada

The Cascade Systems have 2 (or more) refrigeration cycles operating in series.

Cascade cycle is used where a `very` **wide** range of temperature between $T_L$ and $T_H$`is required. 

As shown in the Fig, the `condenser` for the `low` temperature refrigerator is used as the `evaporator` for the `high` temperature refrigerator

![cascade cycle](./img/vcr/2stage_cascade.jpg)

![ts cascade cycle](./img/vcr/2stage_cascade_ts.jpg)

Consider a two-stage cascade refrigeration system operating between pressure limits of $0.8$ and $0.14$ MPa.

Each stage operates on an ideal vapor-compression refrigeration cycle
with refrigerant $R134a$ as working fluid. 

`Heat rejection` from the `lower` cycle to the `upper` cycle takes place in an adiabatic counter flow heat exchanger where both streams enter at
about $0.32$ MPa.

If the mass flow rate of the refrigerant through the `upper` cycle is $0.05$ kg/s,

**Determine**

* a) the mass flow rate of the refrigerant through the lower cycle
* b) the rate of heat removal from the refrigerated space
* c) the power input to the compressors
* d) COP.



## 2 The Analysis

**The mass flow rate is not known in the lower cycle**


### 2.1 The JOSN of Cycles

* vcr_cascade_lower.json

* vcr_cascade_upper.json
 



In [ ]:
# %load ../../SimVCCE/playground/jsonmodel/vcr_cascade_lower.json
{
    "name": "The_Cascade_Lower",
    "refrigerant": "R134a",
    "components": [
        {
            "name": "Compressor",
            "devtype": "COMPRESSOR",
            "iPort": {
            },
            "oPort": {
            },
            "ef": 1.0
        },
        {
            "name": "Condenser",
            "devtype": "CONDENSER",
            "iPort": {},
            "oPort": {
                "x": 0.0,
                "p": 0.32
            }
        },
        {
            "name": "ExpansionValve",
            "devtype": "EXPANSIONVALVE",
            "iPort": {},
            "oPort": {}
        },
        {
            "name": "Evaporator",
            "devtype": "EVAPORATOR",
            "iPort": {},
            "oPort": {
                "p": 0.14,
                "x": 1.0
            }
        }
    ],
    "connectors": [
        [
            {
                "devname": "Compressor",
                "port": "oPort"
            },
            {
                "devname": "Condenser",
                "port": "iPort"
            }
        ],
        [
            {
                "devname": "Condenser",
                "port": "oPort"
            },
            {
                "devname": "ExpansionValve",
                "port": "iPort"
            }
        ],
        [
            {
                "devname": "ExpansionValve",
                "port": "oPort"
            },
            {
                "devname": "Evaporator",
                "port": "iPort"
            }
        ],
        [
            {
                "devname": "Evaporator",
                "port": "oPort"
            },
            {
                "devname": "Compressor",
                "port": "iPort"
            }
        ]
    ]
}

### 2.2 The Main Application
  
The analysis step:

1. the upper cycle
2. **cascade:** lower condenser(Qout) -> upper evaporator(Qin)
3. the lower cycle
4. the performance of the cascade cycle


In [ ]:
# %load ../../SimVCCE/playground/demo_cascade.py

"""
General Object-oriented Abstraction of VC Cycle 
  
  
 The cascade Cycle
 
 lower condenser(Qout) -> upper evaporator(Qin)

 
 Author: Cheng Maohua cmh@seu.edu.cn
"""

import sys
import os
curpath = os.path.abspath(os.path.dirname(__file__))
sys.path.append(curpath+'/../vccpython/')


from vcc.vccobj import VCCycle
from vcc.utils import OutFiles, create_dictcycle_from_jsonfile
from components.port import Port
# new components
from ncomponents.insulated_duct import Insulated_Duct

from platform import os

if __name__ == "__main__":
    curpath = os.path.abspath(os.path.dirname(__file__))
    
    # 1 VC heat pump 
    json_filename = curpath+'\\'+'./jsonmodel/vcr_cascade_upper.json'
    
    thedictcycle_upper = create_dictcycle_from_jsonfile(json_filename)
    cycle_upper = VCCycle(thedictcycle_upper)
    cycle_upper.simulator()
    OutFiles(cycle_upper)

    json_filename_lower = curpath+'\\'+'./jsonmodel/vcr_cascade_lower.json'
    thedictcycle_lower = create_dictcycle_from_jsonfile(json_filename_lower)
    # cascade: lower condenser(Qout) -> upper evaporator(Qin)
    for device in thedictcycle_lower["components"]:
        if device["name"] == "Condenser":
           device["Qout"] = cycle_upper.Qin
           break 
    cycle_lower = VCCycle(thedictcycle_lower)
    cycle_lower.simulator()
    OutFiles(cycle_lower)
   
    Wc = cycle_upper.Wc+cycle_lower.Wc
    cop = cycle_lower.Qin/Wc   
    
    print("\nThe Cascade Refrigeration Cycle")
    print("\tThe mass flow rate of the lower cycle(kg/s): {:.4f}".format(cycle_lower.conns.nodes[0][0].mdot))
    print("\tRefrigeration Capacity(kW): {: .2f}".format(cycle_lower.Qin))
    print("\tHeat Transfer Rate(kW): {: .2f}".format(cycle_upper.Qout))
    print("\tWc(kW):{:.2f}".format(Wc))
    print("\tCOP:{:.2f}".format(cop))
   